# IO Tutorial

This script demonstrates how to load data in pynapple.

The general workflow of loading a session is described by the infographic below. As it is challenging to accomodate all possible types of format, we aimed to keep the IO of pynapple minimal while allowing the user to inherit the base loader and import their own custom io functions. The base loader is thus responsible for initializing the NWB file containing the tracking data, the epochs and the session informations. An example of a custom IO class is shown at the end of this tutorial.


<!-- <img src="images/base_loader_pynapple.png" width=700 height=700 /> -->

![title](base_loader_pynapple.png)


In [1]:
import numpy as np
import pandas as pd
import pynapple as nap
import matplotlib.pyplot as plt

In this [example dataset](https://www.dropbox.com/s/1kc0ulz7yudd9ru/A2929-200711.tar.gz?dl=1), the data contains a sample recording from the anterodorsal nucleus of the thalamus and the hippocampus, with both a sleep and a wake phase. It contains both head-direction cells (i.e. cells that fire for a particular direction of the head in the horizontal plane) and place cells (i.e. cells that fire for a particular position in the environment).

Preprocessing of the data was made with [Kilosort 2.0](https://github.com/MouseLand/Kilosort) and spike sorting was made with [Klusters](http://neurosuite.sourceforge.net/).

A typical directory containing data processed with neurosuite is similar to this:

<img src="files_neurosuite.png" width=700 height=700 />


The first step is to call the function [*load_session*](https://peyrachelab.github.io/pynapple/io/). It will then open a GUI for filling manually the information. The following screenshots show what parameters to use.

In [2]:
data_directory = '../../your/path/to/A2929-200711'

data = nap.load_session(data_directory, 'neurosuite')

<img src="Screenshot1_baseloader.png" width=700 height=700 />


The second step is to provides information about the session and the subject. All the fields shown are suggested by the NWB format (see [here](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) and [here](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile)).

---

The epochs tab loads the epochs within the session (typically wake and sleep). In this case, we load the file Epoch_Ts.csv in the data folder. The first column contains the start of the epoch and the second column contains the end of the epoch.

If the CSV file contains a third column with an epoch label, the loader will automatically write it in the label column. Otherwise, it is necessary to manually write the epoch labels.

<img src="Screenshot4_baseloader.png" width=700 height=700 />

---

The tracking tab allows to load tracking data saved with a CSV file. Reading a CSV file is always a challenge when the header is unknown. The default csv file should contains only one row for the header with the column names. The first column should be the time index in seconds. Other formats are DeepLabCut and Optitrack.

Frame alignement can vary as well. Pynapple offers three ways to align the tracking frames :
- Global timestamps: the time column of the CSV file contains the timestamps aligned to the global timeframe of the session.
- Local timestamps: the time column of the CSV file contains the timestamps aligned to one epoch. In this case, the user should select which epoch.
- TTL detection: a binary file containing TTL pulses for each tracking frame is located within the folder and can be loaded. Alignement is made with TTL detection.

In this example session, Tracking was made with Optitrack and TTL pulses were written to an analogin file recorded by an Intan RHD2000 recording system. The parameters for the tracking tab are shown below.

<img src="Screenshot5_baseloader.png" width=700 height=700 />

---

The next step is specific to NeuroSuite. In this case, 2 electrophysiological probes were implanted, one to the ADN and another to the CA1. This step allows to label groups of electrodes as shown below.

<img src="Screenshot7_baseloader.png" width=700 height=700 />

If successful, a NWB file should be created in session_folder/pynapplenwb/session_name.nwb

<img src="Screenshot8_finale.png" width=700 height=700 />

Calling the function *load_session* should directly read the NWB file and bypass the GUI loader.

In [3]:
data = nap.load_session(data_directory, 'neurosuite')

In this case, the data that can be used for analysis are *spikes*, *position* and *epochs*.

In [4]:
spikes = data.spikes
position = data.position
epochs = data.epochs

print(spikes, '\n')
print(position, '\n')
print(epochs, '\n')

  Index    Freq. (Hz)    group
-------  ------------  -------
      0          7.3         0
      1          5.73        0
      2          8.12        0
      3          6.68        0
      4         10.77        0
      5         11           0
      6         16.52        0
      7          2.2         1
      8          2.02        1
      9          1.07        1
     10          3.92        1
     11          3.31        1
     12          1.09        1
     13          1.28        1
     14          1.32        1 

                  rx        ry        rz         x         y         z
Time (s)                                                              
670.64070   0.343163  5.207148  5.933598 -0.042857  0.425023 -0.195725
670.64900   0.346745  5.181029  5.917368 -0.043863  0.424850 -0.195110
670.65735   0.344035  5.155508  5.905679 -0.044853  0.424697 -0.194674
670.66565   0.322240  5.136537  5.892457 -0.045787  0.424574 -0.194342
670.67400   0.315836  5.120850  5.891577 -0.0

--- 

## Custom Loading IO

This example shows how to construct a custom IO loader.

In [5]:
# -*- coding: utf-8 -*-
# @Author: gviejo
# @Date:   2022-01-06 20:01:32
# @Last Modified by:   gviejo
# @Last Modified time: 2022-01-06 20:01:57

import os
from pynapple.io.loader import BaseLoader
from pynwb import NWBFile, NWBHDF5IO

class MyCustomIO(BaseLoader):

    def __init__(self, path):
        """        
        
        Parameters
        ----------
        path : str
            The path to the data.
        """     
        self.basename = os.path.basename(path)
        
        super().__init__(path)

        # Need to check if nwb file exists and if data are there
        loading_my_data = True
        if self.path is not None:
            nwb_path = os.path.join(self.path, 'pynapplenwb')
            if os.path.exists(nwb_path):
                files = os.listdir(nwb_path)
                if len([f for f in files if f.endswith('.nwb')]):                    
                    success = self.load_my_nwb(path)
                    if success: loading_my_data = False

        # Bypass if data have already been transfered to nwb
        if loading_my_data:
            self.load_my_data(path)

            self.save_my_data_in_nwb(path)

    def load_my_data(self, path):
        """
        This load the raw data
        
        Parameters
        ----------
        path : str
            Path to the session
        """
        '''
        Load Raw data here
        '''
        print(path)
        return None

    def save_my_data_in_nwb(self, path):
        """
        Save the raw data to NWB
        
        Parameters
        ----------
        path : TYPE
            Description
        """
        self.nwb_path = os.path.join(path, 'pynapplenwb')
        if os.path.exists(self.nwb_path):
            files = os.listdir(self.nwb_path)
        else:
            raise RuntimeError("Path {} does not exist.".format(self.nwb_path))
        self.nwbfilename = [f for f in os.listdir(self.nwb_path) if 'nwb' in f][0]
        self.nwbfilepath = os.path.join(self.nwb_path, self.nwbfilename)


        io = NWBHDF5IO(self.nwbfilepath, 'r+')
        
        '''
        Save data in NWB here
        '''

        io.close()

        return
        


    def load_my_nwb(self, path):
        """
        This load the nwb that is already create by the base loader
        
        Parameters
        ----------
        path : str
            Path to the session
        """
        self.nwb_path = os.path.join(path, 'pynapplenwb')
        if os.path.exists(self.nwb_path):
            files = os.listdir(self.nwb_path)
        else:
            raise RuntimeError("Path {} does not exist.".format(self.nwb_path))
        self.nwbfilename = [f for f in os.listdir(self.nwb_path) if 'nwb' in f][0]
        self.nwbfilepath = os.path.join(self.nwb_path, self.nwbfilename)

        io = NWBHDF5IO(self.nwbfilepath, 'r')
        nwbfile = io.read()

        '''
        Add code to write to nwb file here
        '''

        io.close()

            
mydata = MyCustomIO('.')

print(type(mydata))

.
<class '__main__.MyCustomIO'>
